<center><h1 style="font-size: 40px"> Дообученная модель ruBert для задачи экстрактной суммаризации текстов.</h1></center>

# Содержание
### 1. [Дообучение ruBert](#chapter1)
#### 1.1. [Загрузка модели](#chapter1.1)
#### 1.2. [Загрузка и предобработка датасета](#chapter1.2)
#### 1.3. [Дообучение модели ruBert](#chapter1.3)
#### 1.4. [Тестирование дообученной модели](#chapter1.4)

### 2. [Суммаризация текста](#chapter2)
#### 2.1. [Использование библиотеки summarizer](#chapter2.1)
#### 2.2. [Подсчёт всех метрик](#chapter2.2)

### 3. [Вывод](#chapter3)

In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from transformers import BertModel, BertForSequenceClassification, BertTokenizer, AutoConfig

In [4]:
import warnings
warnings.filterwarnings("ignore")

<center id="chapter1"><h1 style="font-size: 24px"> 1. Дообучение ruBert </h1></center>

## 1.1. Загрузка модели <a id="chapter1.1"></a>

In [5]:
model_name = 'DeepPavlov/rubert-base-cased'

tokenizer = BertTokenizer.from_pretrained(model_name)

custom_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2, output_hidden_states=True).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
custom_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

## 1.2. Получение и предобработка данных <a id="chapter1.2"></a>

In [7]:
# Исходный датасет для задачи экстрактной суммаризации
dataset = pd.read_pickle("../Data/ru_train.pkl")
dataset

,sentences,labels
0,"[Современных людей обвиняют в том, что они уби...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[Когда Клэр Хайнс узнала, что беременна, она о...","[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,[Знаменитый шеф-повар Пит Эванс потерял еще од...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ..."
3,"[Ученые полагают, что жизнь на Земле начала ра...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,[Паундленд был вынужден убрать со своих полок ...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
10995,"[Тренер сборной Англии Рой Ходжсон опасается, ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
10996,[Лучшие британцы встретятся лицом к лицу в суб...,"[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10997,"[Шведская футбольная команда рассказала, как о...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10998,"[Лес Фердинанд признал, что перед КПР стоит не...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]"


In [8]:
def make_classification_df(dataset):
    """
    Преобразует исходный датасет в формат, подходящий для решения задачи бинарной классификации.
    """
    train_df = pd.DataFrame()
    
    sentences = []
    labels = []

    for i in dataset.index:
        sentences += list(dataset.loc[i]["sentences"])
        labels += list(dataset.loc[i]["labels"])

    train_df["sentence"] = sentences
    train_df["label"] = labels

    return train_df

In [9]:
# Создадим датасет бинарной классификации на части текстов для начала
df = make_classification_df(dataset)
df

,sentence,label
0,"Современных людей обвиняют в том, что они убил...",0
1,"Но новые исследования показывают, что на самом...",1
2,"Ученые, изучающие новейшие генетические, ископ...",0
3,"Прокрутите вниз, чтобы увидеть видео.",0
4,"Неандертальцы, как и реконструкция, представле...",0
...,...,...
359318,"«Оглядываясь назад, можно сказать, что все отч...",0
359319,"«Англия застала нас врасплох, шла впереди, хор...",0
359320,"Нам было трудно принять это решение, но теперь...",0
359321,"«Пришел даже Мэтт Ричи, он такой большой талант.",0


In [10]:
# В датасете имеется дисбаланс классов. Возьмём одинаковое количество нулевого и первого класса.
df_1 = df[df['label']==1]
df_0 = df[df['label']==0].head(len(df_1))

df_balanced = pd.DataFrame(pd.concat([df_0, df_1]).sample(frac=1).values, columns=['sentence', 'label'])
display(df_balanced)

,sentence,label
0,"Суд услышал, что генеральный секретарь Dignita...",1
1,"Кампала, Уганда (CNN) Боевики на мотоцикле зас...",1
2,"Лишь четверть говорят, что важно знать о внутр...",1
3,"Пол Скоулз (в центре) считает, что «Барселона»...",1
4,«Эта личная катастрофа была полностью вызвана ...,0
...,...,...
57895,Женщины платят меньше: на прежнем месте женщин...,1
57896,"«Охра, возможно, является единственным очевидн...",0
57897,"По ее словам, он также попросил ее поцеловать.",0
57898,"Сегодня мир стал лучше, и одна маленькая девоч...",0


In [11]:
# Пусть обучающая часть содержит 80% предложений, а в валидационной и тестовой оставим по 10%
df_train = df_balanced.loc[0:int(len(df_balanced)*0.8)]
df_val = df_balanced.loc[int(len(df_balanced)*0.8)+1:int(len(df_balanced)*0.9)]
df_test = df_balanced.loc[int(len(df_balanced)*0.9):]

In [12]:
display(df_train)
display(df_val)
display(df_test)

,sentence,label
0,"Суд услышал, что генеральный секретарь Dignita...",1
1,"Кампала, Уганда (CNN) Боевики на мотоцикле зас...",1
2,"Лишь четверть говорят, что важно знать о внутр...",1
3,"Пол Скоулз (в центре) считает, что «Барселона»...",1
4,«Эта личная катастрофа была полностью вызвана ...,0
...,...,...
46316,"Ранее он заявил суду, что «никогда не был скло...",0
46317,Главный тренер «Саутгемптона» Рональд Куман от...,1
46318,(CNN) Попытки некоторых членов Республиканской...,1
46319,"О'Брайен, рост которого почти шесть футов, при...",1


,sentence,label
46321,Но новая книга «Художники и их кошки» возвраща...,1
46322,"Секстинг «не рекомендуется политикам», говорит...",1
46323,Окаменелый скелет (вверху) был найден в скале ...,1
46324,Извинения: Детектив Патрик Черри появляется се...,0
46325,Шон Мэлони проделал 5000-мильную поездку из Чи...,1
...,...,...
52106,Любительница животных: Аманда Холден будет вес...,1
52107,"Когда она открыла дверь такси, в комнату вскоч...",1
52108,"Сообщается, что за 21-летнего аргентинца запла...",1
52109,Бывший полузащитник «Манчестер Юнайтед» был вм...,1


,sentence,label
52110,Фонд поддержки премьер-министра уже покрывает ...,1
52111,Шотландия вылетела из игры со счетом 130 всего...,1
52112,"«Я знаю, что вы отдали свои кровно заработанны...",0
52113,"Температура воды, уровень прилива, мотивация р...",0
52114,Визажист Мария Мэлоун-Гербаа (слева) только чт...,1
...,...,...
57895,Женщины платят меньше: на прежнем месте женщин...,1
57896,"«Охра, возможно, является единственным очевидн...",0
57897,"По ее словам, он также попросил ее поцеловать.",0
57898,"Сегодня мир стал лучше, и одна маленькая девоч...",0


In [13]:
class TextDataset(Dataset):
    """Класс для преобразования датасета к нужному формату"""
    def __init__(self, dataframe, tokenizer, max_len=64):
        self.tokenizer = tokenizer
        self.sentences = dataframe['sentence'].tolist()
        self.labels = dataframe['label'].tolist()
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        text = self.sentences[idx]
        label = self.labels[idx]
        # токенизируем
        inputs = self.tokenizer.encode_plus(
            text=text, # наши данные
            text_pair=None, # это для задачи вопросно-ответной системы, т.е. не для нас
            add_special_tokens=True, # добавление спец-токенов, отвечающих за "начало предложения" [CLS] и "конец предложения" [SEP]
            max_length=self.max_len, # максимальная длина последовательности
            padding='max_length', # если в предложении меньше 64 токенов, то остальные заменяем на пустые
            truncation=True, # если в предложениее 64+ токенов, то мы просто обрезаем их
            return_token_type_ids=False, # это для задачи вопросно-ответной системы, т.е. не для нас
            return_attention_mask=True, # это говорит нашей модели, какие токены важны, а какие просто как padding или [CLS] и т.д.
            return_tensors='pt' # формат выдачи токенизатора, в нашем случае - torch тензор
        )

        # то что мы запихнем в модель
        return {
            'input_ids': inputs['input_ids'].flatten(), # это наши цифровые токены (т.е. для токена 'привет' будет какое-нибудь '105')
            'attention_mask': inputs['attention_mask'].flatten(), # это наши маски
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [16]:
train_dataset = TextDataset(df_train, tokenizer)
val_dataset = TextDataset(df_val, tokenizer)
test_dataset = TextDataset(df_test, tokenizer)

batch_size = 32

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 1.3. Дообучение ruBert <a id="chapter1.3"></a>

In [17]:
import tqdm
from tqdm.auto import tqdm

def train_val(model, train_data_loader, val_data_loader, loss_fn, optimizer, device, num_epochs):
    for t in tqdm(range(num_epochs)):
        train_epoch_loss = []
        eval_epoch_loss = []
        
        print("Эпоха номер: ", t)

        # Обучение модели на текущей эпохе
        model.train()
        for train_data in tqdm(train_data_loader):
            input_ids = train_data['input_ids'].to(device) # токены
            attention_mask = train_data['attention_mask'].to(device) # маски
            labels = train_data['labels'].to(device) # класс

            outputs = model(input_ids=input_ids, attention_mask=attention_mask) # результат модели
            #_, preds = torch.max(outputs.logits, dim=1)
            
            loss = loss_fn(outputs.logits, labels) # считаем потерю
            train_epoch_loss.append(loss.item())


            # Выполним подсчёт новых градиентов
            loss.backward()
            # Выполним шаг градиентного спуска
            optimizer.step()
            # Обнулим сохраненные у оптимизатора значения градиентов
            # перед следующим шагом обучения
            optimizer.zero_grad()

        # Оценка модели на валидационных данных после обучения на текущей эпохе
        model.eval()
        for val_data in tqdm(val_data_loader):
            input_ids = val_data['input_ids'].to(device) # токены
            attention_mask = val_data['attention_mask'].to(device) # маски
            labels = val_data['labels'].to(device) # класс


            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask) # результат модели
                #_, preds = torch.max(outputs.logits, dim=1)
            
                loss = loss_fn(outputs.logits, labels) # считаем потерю
                eval_epoch_loss.append(loss.item())

        # Выведем результаты прошедшей эпохи обучения
        print("Train loss: ", np.mean(train_epoch_loss))
        print("Eval loss: ", np.mean(eval_epoch_loss))
        
    return model

In [18]:
loss_fn = torch.nn.CrossEntropyLoss().to(device)

from transformers import AdamW
optimizer = AdamW(custom_model.parameters(), lr=1e-5)

num_epochs = 3

In [19]:
custom_model = train_val(custom_model, train_data_loader, val_data_loader, loss_fn, optimizer, device, num_epochs)

  0%|          | 0/3 [00:00<?, ?it/s]

Эпоха номер:  0


  0%|          | 0/1448 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
torch.save(custom_model.state_dict(), "custom_rubert_clf.pth")

## 1.4. Тестирование дообученной модели <a id="chapter1.4"></a>

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2, output_hidden_states=True).to(device)
model.load_state_dict(torch.load("custom_rubert_clf.pth"))

In [ ]:
def test_predict(model, test_data_loader):
    test_labels = []
    test_predictions = [] 
    
    for test_data in tqdm(test_data_loader):
        input_ids = test_data['input_ids'].to(device) # токены
        attention_mask = test_data['attention_mask'].to(device) # маски
        labels = test_data['labels'].to(device) # класс

        test_labels += labels.tolist()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask) # результат модели
        _, preds = torch.max(outputs.logits, dim=1)

        test_predictions += preds.tolist()

    print(f1_score(test_labels, test_predictions))
    
    return test_predictions

In [ ]:
with torch.no_grad():
    df_test["predictions"] = test_predict(custom_model, test_data_loader)

<center id="chapter2"><h1 style="font-size: 24px"> 2. Суммаризация текста </h1></center>

Наш датасет:

In [ ]:
extsum_dataset = pd.read_pickle("../Data/ru_test_5k.pkl")
extsum_dataset = pd.DataFrame(extsum_dataset[['ru_src', 'labels']].values, columns=['sentences', 'labels'])
# Взял небольшое количество для проверки кода
extsum_dataset_sample = extsum_dataset.loc[0:10]
extsum_dataset_sample

Наша кастомная модель:

In [ ]:
custom_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2, output_hidden_states=True).to(device)
custom_model.load_state_dict(torch.load("custom_rubert_clf.pth"))

## 2.1. Использование библиотеки summarizer <a id="chapter2.1"></a>

In [ ]:
from summarizer import Summarizer

In [ ]:
bertsum_custom_model = Summarizer(custom_model=custom_model, custom_tokenizer=tokenizer)

In [ ]:
text = " ".join(extsum_dataset_sample.loc[0]["sentences"])
text

In [ ]:
bertsum_custom_model(text, num_sentences=3)

## 2.2. Подсчёт всех метрик <a id="chapter2.2"></a>

Наши предыдущие результаты:

In [ ]:
final_results = pd.read_csv("final_results.csv", index_col=0)
final_results

Наши метрики:

In [ ]:
import evaluate

In [ ]:
rouge = evaluate.load("rouge")
bleu  = evaluate.load("bleu")
meteor = evaluate.load('meteor')

bertscore  = evaluate.load("bertscore")

In [ ]:
def get_metrics(generated_summaries, reference_summaries):
    
    metrics_dict = rouge.compute(predictions=generated_summaries, references=reference_summaries)
    metrics_dict["bleu"] = bleu.compute(predictions=generated_summaries, references=reference_summaries)["bleu"]
    metrics_dict["meteor"] = meteor.compute(predictions=generated_summaries, references=reference_summaries)["meteor"]

    bertscore_results = bertscore.compute(predictions=generated_summaries, references=reference_summaries, lang='ru')
    
    bert_dict = {'bert_precision': np.mean(bertscore_results['precision']), 
                 'bert_recall': np.mean(bertscore_results['recall']),
                'bert_f1': np.mean(bertscore_results['f1'])}
    
    return metrics_dict | bert_dict

In [ ]:
def get_reference_summary(sentences, labels):
    """ 
    Получение образцовой экстрактивной суммаризации по массиву предложений и по маске входящих в суммаризацию предложений
    Параметры:
        sentences - массив строк(предложений)
        labels - маска предложений, 1 - предложение входит в суммаризацию, 0 - не входит
    Возвращает:
        строку
    """
    labels = [x==1 for x in labels]
    
    reference_summary = np.array(sentences)[labels]

    return ' '.join(reference_summary)

In [ ]:
# В этом датафрейме будет два столбца:
# 1. References - эталонные суцммаризации
# 2. CustomBertSum - сгенерированные суммаризации 
df_results = pd.DataFrame()

# Заполняем столбец References
df_results["References"] = extsum_dataset_sample[['sentences', 'labels']].apply(lambda cols: get_reference_summary(cols['sentences'], cols['labels']) , axis=1)
df_results

In [ ]:
# Суммаризируем все тексты
df_results['CustomBertSum'] = extsum_dataset_sample[['sentences', 'labels']].apply(lambda cols: bertsum_custom_model(' '.join(cols['sentences']), num_sentences=sum(cols['labels'])),
                                                          axis=1)

In [ ]:
# Cохраним на всякий полученные суммаризации
df_results.to_pickle("custom_bert_summarizations.pkl")

In [ ]:
# Посчитаем все метрики
metrics_dict = get_metrics(df_results['CustomBertSum'].values, df_results["References"].values)
metrics_dict

In [ ]:
final_results.loc["Custom Bert"] = metrics_dict
final_results

<center id="chapter3"><h1 style="font-size: 24px"> 3. Вывод </h1></center>

Окончательный выбор лучшей модели! Ура!